# Text Generation with Neural Networks

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
import time

2024-04-12 20:04:35.157642: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 20:04:35.867865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Get Text Data

This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [2]:
path_to_file = '../../data/shakespeare.txt'
text = open(path_to_file, 'r').read()

In [3]:
print('First 600 chars: \n')
print(text[:600])

First 600 chars: 


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else th


## Preparing textual data

We need to encode our data to give the model a proper numerical representation of our text.

In [4]:
# creates a set of unique characters found in the text
vocab = sorted(set(text))
# vocab

### Text Vectorization

In [5]:
char_to_int = {u:i for i, u in enumerate(vocab)}
# assigns a unique integer to each character in a dictionary format, 
# creating a mapping that can later be used to transform encoded predictions back into characters
# char_to_int

In [6]:
int_to_char = np.array(vocab)
# reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers, which is used to encode the text.
# int_to_char

In [7]:
encoded_text = np.array([char_to_int[c] for c in text])
# encodes the entire text as an array of integers, with each integer representing the character at that position
#in the text according to the encoder dictionary
# encoded_text

## Creating Training Batches

Training batches are a way of dividing the dataset into smaller, manageable groups of data points that are fed into a machine learning model during the training process.

In [8]:
seq_len = 120 # length of sequence for a training example
total_num_seq = len(text)//(seq_len+1) # total number of training examples

# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

2024-04-12 20:04:38.227894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 20:04:38.267559: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 20:04:38.267614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 20:04:38.278338: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 20:04:38.278468: I tensorflow/compile

In [9]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [10]:
dataset = sequences.map(create_seq_targets)

In [11]:
# Batch size
batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## Creating the GRU Model

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64
# Number of RNN units
rnn_neurons = 1026

In [13]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [14]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

## Instance of the Model

In [15]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [16]:
# TensorBoard
log_dir = "/phoenix/tensorboard/tensorlogs"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

## Training the model

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


2024-04-12 20:04:42.736393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
epochs = 20
model.fit(dataset,epochs=epochs, callbacks=[tensorboard_callback])

Epoch 1/20


2024-04-12 20:04:45.557470: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2a9c0fe0f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-12 20:04:45.557592: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P6000, Compute Capability 6.1
2024-04-12 20:04:45.573751: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-12 20:04:45.732373: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


351/351 [==============================] - 35s 90ms/step - loss: 2.5466
Epoch 2/20
351/351 [==============================] - 33s 90ms/step - loss: 1.7250
Epoch 3/20
351/351 [==============================] - 33s 91ms/step - loss: 1.4528
Epoch 4/20
351/351 [==============================] - 33s 89ms/step - loss: 1.3354
Epoch 5/20
351/351 [==============================] - 32s 88ms/step - loss: 1.2726
Epoch 6/20
351/351 [==============================] - 32s 88ms/step - loss: 1.2329
Epoch 7/20
351/351 [==============================] - 33s 90ms/step - loss: 1.2029
Epoch 8/20
351/351 [==============================] - 33s 91ms/step - loss: 1.1789
Epoch 9/20
351/351 [==============================] - 32s 87ms/step - loss: 1.1592
Epoch 10/20
351/351 [==============================] - 32s 86ms/step - loss: 1.1421
Epoch 11/20
351/351 [==============================] - 31s 86ms/step - loss: 1.1262
Epoch 12/20
351/351 [==============================] - 32s 86ms/step - loss: 1.1118
Epoch 13/20


## Saving the Model

In [20]:
model_name = 'tf_rnn_model.h5'

In [21]:
model.save(f'models/{model_name}') 

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Load Model

In [23]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights(f'models/{model_name}')

model.build(tf.TensorShape([1, None]))

# Generating Predictions

In [24]:
def generate_text(model, start_seed="The ", gen_size=100, temp=1.0):

  num_generate = gen_size
  input_eval = [char_to_int[s] for s in start_seed]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = temp

  model.reset_states()

  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(int_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

#### Generating a text with 1000 chars starting with word 'Confidence'

In [25]:
print(generate_text(model, start_seed="Confidence ", gen_size=1000))

Confidence I have sworn and stay.
  VIOLA. Alas, upon semblancer starve, remains of thy
    sorrows in old plot. And by and by anlike success?
    RIWHERDORE. By days.
  CAMILLO. Nay, let your knee speak.
  GREMIO. To see how to lovice! [Wilt anon] LUCIUS, earl of Scotland
  NYMufflew,
    With silver all despite. Trumpet, welcome, doom;
    Good Humbham, and none of my sight.
    To me relieve the Empress's brow,
    And bear his lets grow landica amazes,
    And she appeaised sheep; and welcome within.
                                 [Laesellowed woman to Such enter a fox. Placonius,
    Give mey death hath the infamy blue,
    And let what curious angel be usure's death!
    I have not spoken his ambition
    I bear this treesies which here comes too.
    I tarry with our holy kinsman.  
  Though ope of industry, stay, look!
    How long is for some meat the wager?
    Do not do for a stolan here shall shed.
  BASSANIO. Truth, my lord, I thank thee for their children's death.
  LUC

#### Generating a text with 1000 chars starting with word 'Love'

In [26]:
print(generate_text(model, start_seed="Love ", gen_size=1000))

Love the land to men.
    Your valiant Claudio's partial. Twine foil in
    the over, she shall be cannot live.
  GLOUCESTER. Return again before thy love to-morrow, sure!
    What maliff'd with our tender lecture sament,
    Draws beauty out.
  AJAX. No, sirrah, and therein as he is. Thou'rt mine,
    Yout'-put Henry's right he yet look and
    Bashful be dypar'd? All, 'tis crown'd?
    Your poresal distending honesty.
  IThe is not the most
    of poor Earl of Demetrius, fill this fashion of Hercules
    To o'er-convenient friendly come to this?
    A plagues, whom they are rav'd but strange
    When he's season to serve for me; I mean the first
    And ta'en upon this dearny.
  MENELAUS. Here's the trick o' their story remedies
    To one, or a suitor turns it out the letter?
  BUCKINGHAM. Now, hear Herself, as our regard thereof
    With objaccedor the complain of the sun.
  THESEUS. What, are you of my ld reason to bed-
    She shall be so.
I'ld hold his complaining where they mee